
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [185]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
from scipy import stats


In [186]:
# Choose which results to use
results_to_use = ["Random", "Zero"]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['Random', 'Zero']


In [187]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [188]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: Random
                                    0                   1
0                             command                 run
1                       num_of_epochs               10000
2                  num_of_simulations                  25
3                           grid_size                   3
4                      queue_capacity                  10
5                     congestion_rate                0.07
6                       with_hotspots                True
7                           wage_time                  10
8                      credit_balance                   5
9           shared_auction_parameters                None
10  adaptive_auction_action_selection              random
11               bid_calculation_rule              linear
12       adaptive_auction_update_rule       simple_bandit
13             auction_episode_length                  10
14   action_selection_hyperparameters                None
15    adaptive_auction_discretization                  2

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [189]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=4.363939513446337, pvalue=6.75760432472137e-05, df=48.0)
The average number of trips for the first two simulations are: 24730.08 and 24565.88
The p-value is: 6.75760432472137e-05
The difference is significant


## 2. Average Congestion of Central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [190]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")

TtestResult(statistic=-11.640317243016918, pvalue=1.3950057096818932e-15, df=48.0)
The average congestion for the first two simulations are: 0.3288947 and 0.3448637
The p-value is: 1.3950057096818932e-15
The difference is significant


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [191]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))

TtestResult(statistic=-56.58202279876511, pvalue=1.3436017351289693e-45, df=48.0)
The average time waited for the first two simulations are: 0.27847644444444447 and 0.5983984444444445
The p-value is: 1.3436017351289693e-45
TtestResult(statistic=-54.12898781950441, pvalue=1.0920071927733087e-44, df=48.0)
The max time waited for the first two simulations are: 0.7197417777777778 and 1.6794879999999999
The p-value is: 1.0920071927733087e-44


In [192]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-4.965228849931699, pvalue=9.07432923047412e-06, df=48.0)
The average time waited for the first two simulations are: 0.022799 and 0.028337000000000005
The p-value is: 9.07432923047412e-06
Intersection [ 0 ][ 1 ]
TtestResult(statistic=-20.456521454546372, pvalue=2.397083978200404e-25, df=48.0)
The average time waited for the first two simulations are: 0.24614600000000003 and 0.6458840000000001
The p-value is: 2.397083978200404e-25
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-2.9636654749631033, pvalue=0.004720328101911658, df=48.0)
The average time waited for the first two simulations are: 0.025911 and 0.029918999999999998
The p-value is: 0.004720328101911658
Intersection [ 1 ][ 0 ]
TtestResult(statistic=-20.534894906274232, pvalue=2.032270003206872e-25, df=48.0)
The average time waited for the first two simulations are: 0.316962 and 0.694126
The p-value is: 2.032270003206872e-25
Intersection [ 1 ][ 1 ]
TtestResult(statistic=-56.4024632193

In [193]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        print(results)
        print("The average time waited for the first two simulations are: " + str(np.mean(max_time_intersection[0][:,i,j])) + " and " + str(np.mean(max_time_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=-5.657754957846882, pvalue=8.319328847739101e-07, df=48.0)
The average time waited for the first two simulations are: 0.08212799999999999 and 0.10249199999999999
The p-value is: 8.319328847739101e-07
Intersection [ 0 ][ 1 ]
TtestResult(statistic=-20.245458062921816, pvalue=3.7486566541973935e-25, df=48.0)
The average time waited for the first two simulations are: 0.68946 and 1.9061599999999999
The p-value is: 3.7486566541973935e-25
Intersection [ 0 ][ 2 ]
TtestResult(statistic=-4.5523794058979545, pvalue=3.633097144705971e-05, df=48.0)
The average time waited for the first two simulations are: 0.09016 and 0.107928
The p-value is: 3.633097144705971e-05
Intersection [ 1 ][ 0 ]
TtestResult(statistic=-21.665915024889475, pvalue=1.981669573797869e-26, df=48.0)
The average time waited for the first two simulations are: 0.86638 and 2.047192
The p-value is: 1.981669573797869e-26
Intersection [ 1 ][ 1 ]
TtestResult(statistic=-53.75761424629728, pval

In [194]:
######Grid based######

# Average time
print("AVERAGE TIME WAITED")
average_time_grid = []
for exp in results_to_use:
    average_time_grid.append(
        all_sim_results[exp]['stat_average_time_waited_grid'])

means_1 = []
for i in range(len(average_time_grid[0])):
    means_1.append(np.mean(average_time_grid[0][i]))
means_2 = []
for i in range(len(average_time_grid[1])):
    means_2.append(np.mean(average_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))


# Max time
print("MAX TIME WAITED")
max_time_grid = []
for exp in results_to_use:
    max_time_grid.append(
        all_sim_results[exp]['stat_max_time_waited_grid'])

means_1 = []
for i in range(len(max_time_grid[0])):
    means_1.append(np.mean(max_time_grid[0][i]))
means_2 = []
for i in range(len(max_time_grid[1])):
    means_2.append(np.mean(max_time_grid[1][i]))


results = stats.ttest_ind(means_1, means_2)
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))

AVERAGE TIME WAITED
TtestResult(statistic=-56.58202279876512, pvalue=1.3436017351289312e-45, df=48.0)
The average time waited for the first two simulations are: 0.2784764444444444 and 0.5983984444444443
The p-value is: 1.3436017351289312e-45
MAX TIME WAITED
TtestResult(statistic=-54.1289878195044, pvalue=1.0920071927733087e-44, df=48.0)
The max time waited for the first two simulations are: 0.7197417777777778 and 1.6794879999999999
The p-value is: 1.0920071927733087e-44


## 4. Gini coefficient <a id="4"></a>

In [195]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
# Present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=4.914093418635146, pvalue=1.0794996299799161e-05, df=48.0)
The average gini of time waited for the first two simulations are: 0.13414021539611184 and 0.13124613361181756
The p-value is: 1.0794996299799161e-05
The difference is significant


In [196]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=4.914093418635146, pvalue=1.0794996299799161e-05, df=48.0)
The average gini of satisfaction for the first two simulations are: 0.433927463557328 and 0.44470795444221245
The p-value is: 1.0794996299799161e-05
The difference is significant


## 5. Trip Satisfaction <a id="5"></a>

In [197]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfactino for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=-16.92456209855604, pvalue=7.200842559067493e-22, df=48.0)
The average satisfactino for the first two simulations are: 0.1551592423202917 and 0.15989969447709038
The p-value is: 7.200842559067493e-22
The difference is significant


## 6. Auction Reward <a id="5"></a>

In [198]:
######Intersection based - Average Auction Reward######

average_reward_intersection = []
for exp in results_to_use:
    average_reward_intersection.append(
        all_sim_results[exp]['stat_average_auction_reward_per_intersection'])
    
for i in range(len(average_reward_intersection[0][0])):
    for j in range(len(average_reward_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_reward_intersection[0][:,i,j], average_reward_intersection[1][:,i,j])
        print(results)
        print("The average auction reward for the first two simulations are: " + str(np.mean(average_reward_intersection[0][:,i,j])) + " and " + str(np.mean(average_reward_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=3.845497245791509, pvalue=0.0003540475044749223, df=48.0)
The average auction reward for the first two simulations are: 0.04936466666666666 and 0.045822666666666664
The p-value is: 0.0003540475044749223
Intersection [ 0 ][ 1 ]
TtestResult(statistic=5.639182254259959, pvalue=8.876449945172948e-07, df=48.0)
The average auction reward for the first two simulations are: 0.24603599999999998 and 0.23117633333333332
The p-value is: 8.876449945172948e-07
Intersection [ 0 ][ 2 ]
TtestResult(statistic=3.3845876455163024, pvalue=0.00142844545509015, df=48.0)
The average auction reward for the first two simulations are: 0.05017799999999999 and 0.04694999999999999
The p-value is: 0.00142844545509015
Intersection [ 1 ][ 0 ]
TtestResult(statistic=11.244784269823464, pvalue=4.738830768358722e-15, df=48.0)
The average auction reward for the first two simulations are: 0.2534586666666666 and 0.2276193333333333
The p-value is: 4.738830768358722e-15
Intersectio